In [1]:
import pygame
from pygame.color import THECOLORS
import pymunk
from pymunk.vec2d import Vec2d
from pymunk.pygame_util import draw

Loading chipmunk for Linux (64bit) [/home/rebcabin/anaconda3/envs/self-driving-car/lib/python3.5/site-packages/pymunk/libchipmunk64.so]


In [2]:
width = 1000
height = 700
pygame.init()
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()
screen.set_alpha(None)

In [84]:
def clear_screen(color=THECOLORS['black']):
    screen.fill(color)
    pygame.display.flip()

In [103]:
def draw_puck(p_0=Vec2d(width/2, height/2), puck_color=THECOLORS['red']):
    dont_fill_bit = 0
    clear_screen()
    r = 100
    pygame.draw.circle(screen, puck_color, p_0.int_tuple, r, dont_fill_bit)
    pygame.display.flip()
draw_puck()

In [104]:
def anim_puck(
        steps=100, 
        dt=1, 
        p_0=Vec2d(width/2, height/2), 
        v_0=Vec2d(1, 1/2), 
        puck_color=THECOLORS['red']):
    for i in range(steps):
        clear_screen()
        p_i = p_0 + (i * dt * v_0)
        pygame.draw.circle(screen, puck_color, p_i.int_tuple, r, dont_fill_bit)
        pygame.display.flip()
anim_puck()

## PREDICT WALL STRIKES

In [105]:
import sympy

Function to return a parametric function that produces all the points along a line:

In [106]:
def parametric_line(p0: Vec2d, p1: Vec2d):
    d = (p1 - p0).get_length()
    return lambda t: p0 + t * (p1 - p0) / d

In [107]:
import numpy.random as rndm

In [108]:
def random_points(n):
    return [Vec2d(p[0]*width, p[1]*height) for p in rndm.random((n, 2))]

https://goo.gl/PR24H2
https://goo.gl/x3dEM6

In [109]:
from typing import List, Tuple

In [110]:
def convex_hull(points: List[Vec2d]) -> List[Tuple[int, int]]:
    """Computes the convex hull of a set of 2D points.

    Input: an iterable sequence of Vec2ds(x, y) representing the points.
    Output: a list of integer vertices of the convex hull in counter-clockwise 
    order, starting from the vertex with the lexicographically smallest 
    coordinates. Implements Andrew's monotone chain algorithm. 
    O(n log n) complexity in time."""

    # Sort the points lexicographically (tuples are compared lexicographically).
    # Remove duplicates to detect the case we have just one unique point.
    points = sorted(set([p.int_tuple for p in points]))

    # Boring case: no points or a single point, possibly repeated multiple times.
    if len(points) <= 1:
        return points

    # 2D cross product of OA and OB vectors, i.e. z-component of their 3D cross product.
    # Returns a positive value, if OAB makes a counter-clockwise turn,
    # negative for clockwise turn, and zero if the points are collinear.
    def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

    # Build lower hull 
    lower = []
    for p in points:
        while len(lower) >= 2 and cross(lower[-2], lower[-1], p) <= 0:
            lower.pop()
        lower.append(p)

    # Build upper hull
    upper = []
    for p in reversed(points):
        while len(upper) >= 2 and cross(upper[-2], upper[-1], p) <= 0:
            upper.pop()
        upper.append(p)

    # Concatenation of the lower and upper hulls gives the convex hull.
    # Last point of each list is omitted because it is repeated at the 
    # beginning of the other list. 
    return lower[:-1] + upper[:-1]

In [111]:
def screen_cage(pad=10):
    return [Vec2d(pad, pad),
            Vec2d(pad, height-pad-1),
            Vec2d(width-pad-1, height-pad-1),
            Vec2d(width-pad-1, pad)]

In [113]:
def draw_points(int_tuples: List[Tuple[int, int]],
                color=THECOLORS['yellow']):
    pygame.draw.polygon(screen, color, int_tuples, 1)
    pygame.display.flip()
anim_puck()
# draw_points(convex_hull(random_points(15)))
draw_points([p.int_tuple for p in screen_cage()], THECOLORS['green'])

In [124]:
def colinear_point_and_parameter(u: Vec2d, v: Vec2d, p: Vec2d) -> Tuple[Vec2d, float]:
    vmu_squared = (v - u).dot(v - u)
    t = (p - u).dot(v - u) / vmu_squared
    q = u + t * (v - u)
    return (q, t)
def draw_col_pt_param(u=Vec2d(10, height - 10 - 1),
                      v=Vec2d(width - 10 - 1, height - 10 - 1),
                      p=Vec2d(width/2 + 99, height/2 + 99/2),
                      point_color=THECOLORS['black'],
                      line_color=THECOLORS['magenta']):
    q, t = colinear_point_and_parameter(u, v, p)
    pygame.draw.line(screen, point_color, q.int_tuple, q.int_tuple)
    pygame.draw.line(screen, line_color, p.int_tuple, q.int_tuple)
draw_col_pt_param()

In [115]:
def predict_hit(line_p_0: Vec2d, 
                line_p_1: Vec2d, 
                puck_center: Vec2d=Vec2d(width/2, height/2),
                puck_radius: float=100.0,
                puck_vel: Vec2d=Vec2d(1, 1/2),
                dt: float=1.0):
    pass